In [1]:
#importando as bibliotecas
from scrapy.item import Field
from scrapy.item import Item
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor
from scrapy.loader import ItemLoader
from bs4 import BeautifulSoup
from scrapy.crawler import CrawlerProcess
from itemloaders.processors import MapCompose
import pandas as pd
import json


In [2]:
class Articulo(Item):
    titulo = Field()
    preco = Field()
    descricao = Field()


In [3]:
# 
class MercadoLivreCrawler(CrawlSpider):
    name = 'mercadoLivre'

    custom_settings = {
      'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36',
      'CLOSESPIDER_PAGECOUNT': 10  # Número máximo de páginas a serem baixadas
    }

    allowed_domains = ['mercadolivre.com.br', 'lista.mercadolivre.com.br']
    start_urls = ['https://lista.mercadolivre.com.br/animais/cachorros/brinquedos/']
    download_delay = 1

    rules = (
        Rule(
            LinkExtractor(
                allow=r'/_Desde_\d+'  # Paginação
            ), follow=True),
        Rule(
            LinkExtractor(
                allow=r'/MLB-'  # Detalhes do produto (alterado para o padrão do Brasil)
            ), follow=True, callback='parse_items'),
    )

    def parse_items(self, response):
        item = ItemLoader(Articulo(), response)

        # Extraindo título e descrição
        item.add_xpath('titulo', '//h1/text()', MapCompose(lambda i: i.strip()))
        item.add_xpath('descricao', '//div[@class="ui-pdp-description__content"]/p/text()', MapCompose(lambda i: i.strip()))

        # Extraindo o preço com BeautifulSoup
        soup = BeautifulSoup(response.body, 'html.parser')
        preco = soup.find(class_="andes-money-amount__fraction")
        if preco:
            preco_completo = preco.text.replace('\n', '').replace('\r', '').replace(' ', '')
            item.add_value('preco', preco_completo)

        yield item.load_item()


In [4]:
# Função para salvar em JSON e Excel
def salvar_resultados(resultados):
    # Salvando em JSON
    with open('mercadolivre_brasil.json', 'w', encoding='utf-8') as json_file:
        json.dump(resultados, json_file, ensure_ascii=False, indent=4)

    # Salvando em Excel
    df = pd.DataFrame(resultados)
    df.to_excel('mercadolivre_brasil.xlsx', index=False)

process = CrawlerProcess({
    'FEED_FORMAT': 'json',
    'FEED_URI': 'mercadolivre_brasil.json'
})

resultados = []

# Executar o crawler e coletar os resultados
process.crawl(MercadoLivreCrawler)
process.start()

# Lendo os resultados do arquivo JSON para salvá-los em Excel
resultados = []
with open('mercadolivre_brasil.json', encoding='utf-8') as f:
    for line in f:
        line = line.strip()  # Remove espaços em branco no início e no final da linha
        if line:  # Verifica se a linha não está vazia
            try:
                resultados.append(json.loads(line))
            except json.JSONDecodeError as e:
                print(f"Erro de formatação no arquivo JSON: {e}")

# Chamando a função para salvar os resultados em Excel
salvar_resultados(resultados)

2024-09-07 16:41:32 [scrapy.utils.log] INFO: Scrapy 2.11.2 started (bot: scrapybot)
2024-09-07 16:41:32 [scrapy.utils.log] INFO: Versions: lxml 5.3.0.0, libxml2 2.11.7, cssselect 1.2.0, parsel 1.9.1, w3lib 2.2.1, Twisted 24.7.0, Python 3.12.5 (tags/v3.12.5:ff3bc82, Aug  6 2024, 20:45:27) [MSC v.1940 64 bit (AMD64)], pyOpenSSL 24.2.1 (OpenSSL 3.3.2 3 Sep 2024), cryptography 43.0.1, Platform Windows-10-10.0.19045-SP0
2024-09-07 16:41:32 [scrapy.addons] INFO: Enabled addons:
[]
2024-09-07 16:41:32 [py.warnings] WARNING: c:\Users\cezar\AppData\Local\Programs\Python\Python312\Lib\site-packages\scrapy\utils\request.py:254: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.


2024-09-07 16:41:32 [scrapy.extensions.telnet] INFO: Telnet Password: b0003eb5c382131f
2024-09-07 16:41:34 [py.warnings] WARNING: c:\Users\cezar\AppData\Local\Programs\Python\Python312\Lib\site-packages\scrapy\extensions\feedexport.py:406: ScrapyDeprecationWarning: The `FEED_URI` and `FEED_FORMAT` settings have been deprecated in favor of the `FEEDS` setting. Please see the `FEEDS` setting docs for more details
  exporter = cls(crawler)

2024-09-07 16:41:34 [scrapy.middleware] INFO: Enabled extensions:
['scrapy.extensions.corestats.CoreStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.closespider.CloseSpider',
 'scrapy.extensions.feedexport.FeedExporter',
 'scrapy.extensions.logstats.LogStats']
2024-09-07 16:41:34 [scrapy.crawler] INFO: Overridden settings:
{'CLOSESPIDER_PAGECOUNT': 10,
 'USER_AGENT': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 '
               '(KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}
2024-09-07 16:41:38 [scrapy.middl

Erro de formatação no arquivo JSON: Expecting value: line 1 column 2 (char 1)
Erro de formatação no arquivo JSON: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Erro de formatação no arquivo JSON: Extra data: line 1 column 9 (char 8)
Erro de formatação no arquivo JSON: Expecting value: line 1 column 1 (char 0)
Erro de formatação no arquivo JSON: Extra data: line 1 column 8 (char 7)
Erro de formatação no arquivo JSON: Expecting value: line 1 column 1 (char 0)
Erro de formatação no arquivo JSON: Expecting value: line 1 column 1 (char 0)
Erro de formatação no arquivo JSON: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)
Erro de formatação no arquivo JSON: Extra data: line 1 column 9 (char 8)
Erro de formatação no arquivo JSON: Expecting value: line 1 column 1 (char 0)
Erro de formatação no arquivo JSON: Extra data: line 1 column 8 (char 7)
Erro de formatação no arquivo JSON: Expecting value: line 1 column 1 (char 0)
Erro de formataç